In [15]:
import torch
import huggingface_hub
import utils
from transformers import ViTFeatureExtractor, ViTModel, ViTForImageClassification, ViTImageProcessor

In [2]:
print('List of available models')
print(utils.get_available_models())
print('List of models trained on EGO4D')
indexes_ego = [i for i, x in enumerate(utils.get_available_models()) if 'ego' in x]
print( [utils.get_available_models()[i] for i in indexes_ego])

print('List of models trained on SAYCam')
indexes_say = [i for i, x in enumerate(utils.get_available_models()) if 'say' in x]
print([utils.get_available_models()[i] for i in indexes_say])

print('Only 2 compatible models are thus dino_ego4d-200h_vitb14 and dino_say_vitb14')


List of available models
['dino_say_vitb14', 'dino_s_vitb14', 'dino_a_vitb14', 'dino_y_vitb14', 'dino_sfp_vitb14', 'dino_imagenet100_vitb14', 'dino_imagenet10_vitb14', 'dino_imagenet1_vitb14', 'dino_kinetics-200h_vitb14', 'dino_ego4d-200h_vitb14', 'dino_say_resnext50', 'dino_s_resnext50', 'dino_a_resnext50', 'dino_y_resnext50', 'dino_sfp_resnext50', 'dino_say_vitl16', 'dino_s_vitl16', 'dino_a_vitl16', 'dino_y_vitl16', 'dino_say_vitb16', 'dino_s_vitb16', 'dino_a_vitb16', 'dino_y_vitb16', 'dino_say_vits16', 'dino_s_vits16', 'dino_a_vits16', 'dino_y_vits16', 'mugs_say_vitl16', 'mugs_s_vitl16', 'mugs_a_vitl16', 'mugs_y_vitl16', 'mugs_say_vitb16', 'mugs_s_vitb16', 'mugs_a_vitb16', 'mugs_y_vitb16', 'mugs_say_vits16', 'mugs_s_vits16', 'mugs_a_vits16', 'mugs_y_vits16', 'mae_say_vitl16', 'mae_s_vitl16', 'mae_a_vitl16', 'mae_y_vitl16', 'mae_say_vitb16', 'mae_s_vitb16', 'mae_a_vitb16', 'mae_y_vitb16', 'mae_say_vits16', 'mae_s_vits16', 'mae_a_vits16', 'mae_y_vits16']
List of models trained on EGO4

In [3]:
from utils import *

#model_say = load_model('dino_say_vitb14')
model_ego = load_model('dino_ego4d-200h_vitb14')

Take key teacher in provided checkpoint dict
Pretrained weights found at /data/alban/models/huggingface_hub/models--eminorhan--dino_ego4d-200h_vitb14/snapshots/6222b6a14c40a4dad7d935460da8226a90c7d225/dino_ego4d-200h_vitb14.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])


In [30]:
## Print model
#print(model_say)
print(model_ego)
#model_ego.pos_embed.shape

VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): ModuleList(
    (0-11): 12 x Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
      )
    )
  )
  (norm): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
  (head): Identity()
)


In [17]:
processor = processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model_supervised = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
print(model_supervised)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
            (intermed

In [24]:
# Extract CLS token
# Load image
from PIL import Image
image = Image.open('testsets/100_faces/0001_f_n000009_test_003_0004_01.png')

# Process
inputs = processor(images=image, return_tensors="pt")

# Extract CLS token
model_supervised.eval()
with torch.no_grad():
    outputs = model_supervised(**inputs, output_hidden_states=True)
    cls_token = outputs.hidden_states[-1][:, 0]

outputs.hidden_states[-1].shape

torch.Size([1, 197, 768])

In [28]:
import numpy as np
from os.path import join
path2activations = f'/data/alban/activations/supervised_hmIT_stimuli'
cls_token = list()
#patch_token = list()
cls_token.append(np.load(join(path2activations, f'cls_token_stimulus1.npy')))
cls_token = np.array(cls_token)
cls_token.shape

(1, 12, 1, 768)